In [1]:
import ee
import numpy as np
import requests
import shutil
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from IPython.display import Image
from sklearn.metrics import confusion_matrix

In [2]:
ee.Initialize()

## Load Data

In [3]:
df = pd.read_csv('../pheno-data.csv')
df = df[(df['Phenophase_Description'] == 'Leaves (grasses)') & (df['Intensity_Value'] != '-9999')]
df['Abundance_Binary'] = df['Intensity_Value'].apply(lambda x: 1 if x == '75-94%' or x == '50-74%' or x == '95% or more' else 0)
df_filtered = df[['Observation_ID','Latitude', 'Longitude', 'Phenophase_ID', 'Observation_Date', 'Abundance_Binary']]
df_filtered.head()

,Observation_ID,Latitude,Longitude,Phenophase_ID,Observation_Date,Abundance_Binary
1754,1795824,32.250660,-110.946358,489,2013-01-28,1
1756,6838318,32.250660,-110.946358,489,2015-12-08,1
8332,2588139,32.212666,-111.001831,489,2013-07-23,1
8333,2588163,32.212666,-111.001831,489,2013-07-31,1
8334,2588187,32.212666,-111.001831,489,2013-08-06,1


In [ ]:
scale = 4000 ## meters
label = df_filtered.Abundance_Binary.values
for days in range(1,25):
    pred = []
    for i in tqdm(range(len(df_filtered))):
        
        ## load precipitation data 
        prec = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
        prec = prec.select('total_precipitation_sum')
        
        ## filter date
        start_date = df_filtered.Observation_Date.values[i]
        end_date = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=days)).strftime("%Y-%m-%d")
        prec = prec.filterDate(start_date, end_date)
        
        ## point location
        lat = df_filtered.Latitude.values[i]
        long = df_filtered.Longitude.values[i]
        point = ee.Geometry.Point([long, lat])
        
        ## get precipitation time series data
        try:
            ts_prec = prec.getRegion(point, scale).getInfo()
            ts_prec = [k[-1]*39.3701 for k in ts_prec[1:]] ## 1 meter = 39.3701 inches
            pred.append(1*(sum(ts_prec)>1.7))
        except:
            pred.append(0)
            
    tn, fp, fn, tp = confusion_matrix(pred, label).ravel()            
    acc = 100*(tn+tp)/len(label)
    fp_rate = 100*(fp)/len(label)
    fn_rate = 100*(fn)/len(label)
    
    print(f'Accumulative Days {days}.')
    print(f'Accuracy: {acc}%')
    print(f'FP: {fp_rate}%')
    print(f'FN: {fn_rate}%')

100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [11:06<00:00,  2.55it/s]


Accumulative Days 1.
Accuracy: 64.25206124852768%
FP: 35.74793875147232%
FN: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [10:53<00:00,  2.60it/s]


Accumulative Days 2.
Accuracy: 64.31095406360424%
FP: 35.68904593639576%
FN: 0.0%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [10:54<00:00,  2.59it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [10:32<00:00,  2.68it/s]


Accumulative Days 6.
Accuracy: 64.31095406360424%
FP: 34.51118963486454%
FN: 1.1778563015312131%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [10:05<00:00,  2.81it/s]


Accumulative Days 7.
Accuracy: 64.01648998822144%
FP: 34.33451118963487%
FN: 1.6489988221436984%


 24%|██████████████████▉                                                            | 407/1698 [02:47<07:28,  2.88it/s]

In [4]:
scale = 4000 ## meters
label = df_filtered.Abundance_Binary.values
for days in range(8,25):
    pred = []
    for i in tqdm(range(len(df_filtered))):
        
        ## load precipitation data 
        prec = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
        prec = prec.select('total_precipitation_sum')
        
        ## filter date
        start_date = df_filtered.Observation_Date.values[i]
        end_date = (datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=days)).strftime("%Y-%m-%d")
        prec = prec.filterDate(start_date, end_date)
        
        ## point location
        lat = df_filtered.Latitude.values[i]
        long = df_filtered.Longitude.values[i]
        point = ee.Geometry.Point([long, lat])
        
        ## get precipitation time series data
        try:
            ts_prec = prec.getRegion(point, scale).getInfo()
            ts_prec = [k[-1]*39.3701 for k in ts_prec[1:]] ## 1 meter = 39.3701 inches
            pred.append(1*(sum(ts_prec)>1.7))
        except:
            pred.append(0)
            
    tn, fp, fn, tp = confusion_matrix(pred, label).ravel()            
    acc = 100*(tn+tp)/len(label)
    fp_rate = 100*(fp)/len(label)
    fn_rate = 100*(fn)/len(label)
    
    print(f'Accumulative Days {days}.')
    print(f'Accuracy: {acc}%')
    print(f'FP: {fp_rate}%')
    print(f'FN: {fn_rate}%')

100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [11:45<00:00,  2.41it/s]


Accumulative Days 8.
Accuracy: 63.486454652532395%
FP: 34.275618374558306%
FN: 2.237926972909305%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [11:24<00:00,  2.48it/s]


Accumulative Days 9.
Accuracy: 63.957597173144876%
FP: 33.51001177856301%
FN: 2.532391048292108%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [11:27<00:00,  2.47it/s]


...

Accumulative Days 22.
Accuracy: 60.71849234393404%
FP: 29.44640753828033%
FN: 9.83510011778563%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [11:35<00:00,  2.44it/s]


Accumulative Days 23.
Accuracy: 60.83627797408716%
FP: 28.50412249705536%
FN: 10.659599528857479%


100%|██████████████████████████████████████████████████████████████████████████████| 1698/1698 [11:51<00:00,  2.39it/s]

Accumulative Days 24.
Accuracy: 60.42402826855124%
FP: 28.50412249705536%
FN: 11.071849234393405%
